In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=1)

In [20]:
%%time
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math
   
#... Functions
#_____________

# gets days to expiry from now onwards
def get_dte(dt):
    '''Gets days to expiry
    Arg: (dt) as day in string format 'yyyymmdd'
    Returns: days to expiry as int'''
    return (util.parseIBDatetime(dt) - 
            datetime.datetime.now().date()).days

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
    # if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = get_dte(expiry)
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

def get_prec(v, base):
    '''gives the precision value
    args:
       (v) as value needing precision in float
       (base) as the base value e.g. 0.05'''
    
    return round(round((v)/ base) * base, -int(math.floor(math.log10(base))))

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: df as dataframe
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

def short_opt_margin(contract, mult=1):
    '''Gets margin of a single option contract
    Args:
        (contract) object as the option contract
        <mult> = 1 as int = no of contracts
    Returns: margin as a float'''
    
    order = Order(action='SELL', totalQuantity=mult, orderType='MKT')
    margin = float(ib.whatIfOrder(contract, order).initMarginChange)
    return margin

#...Strategies to filter
#------------------------
def filter_kxdte(df, ohlc):
    '''Filters the strikes by dte*3, and, min of lows for Puts and max of highs for Calls
    Args: 
	   df as dataframe object
	   ohlc as ohlc dataframe object
    Returns: cleansed dfs without the risky strikes'''
    
    df['sfilt'] = [df_ohlc.set_index('symbol').loc[s][:max(min_dte, d*3)].low.min() 
     if g == 'P' 
     else df_ohlc.set_index('symbol').loc[s][:max(min_dte, d*3)].high.max() 
     for s, d, g in zip(df.symbol, df.dte, df.right)]
    
    df = df[((df.right == 'P') & (df.strike < df.sfilt)) | \
            ((df.right == 'C') & (df.strike > df.sfilt))]
    
    # return sorted df of puts and calls    
    df1 = grp_opts(df)
    
    # drop sfilt    
    return df1.drop('sfilt', axis=1)

#-----------------------------

def strat_hilo52(df):
    '''Keeps only options beyond 52 week high and low
    Arg: (df) as dataframe object
    Return: (df_opt) as dataframe with hilo52 options'''
    hilo_mask = ((df.right == 'P') & (df.strike < df.lo52)) | ((df.right == 'C') & (df.strike > df.hi52))
    df_opt = df[hilo_mask].reset_index(drop=True)
    return df_opt

#-----------------------------

def strat_onlyputs(df):
    '''Keep only puts
    Arg: (df) as dataframe object
    Return: (df_opt) without calls'''
    
    return df[df.right=='P'].reset_index()

#...assignments
#______________

m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices
expmult = 1   # 1.05 is 5% expected multiple. Used for expected sowing price

desired_rom = 1.8# desired rom to give the target price.

min_rom = 0.85
min_pop = 1
min_dte = 45     # no of minimum dte days to determine ohlc filter for min and max

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin
    
#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...Harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['symbol'] = [s.symbol for s in df_p.contract.values]

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
strike = [d.strike for d in df_p.contract]

df_p['expiry'] = expiry
df_p['strike'] = strike
df_p['dte'] = [get_dte(d.lastTradeDateOrContractMonth) for d in df_p.contract]

discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_p[df_p.dte > 0].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)

# ...sowing prepration
#_____________________

#... get the lots and margins

tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

# Truncate to 9 characters for ibSymbol
df_tp['symbol'] = df_tp.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_tp.symbol = df_tp.symbol.replace(ntoi)

pos_cols = ['symbol', 'position', 'marketValue', 'unrealizedPNL', 'realizedPNL']
df_pos = df_p[pos_cols]

#... make the blacklist
df1 = df_pos.groupby('symbol').sum()

df1 = df1.join(df_tp[['symbol', 'lot', 'margin']].set_index('symbol'), how='inner')
df1['marginUsed'] =  df1.position/df1.lot*df1.margin

df1['marginAvlbl'] = df1.marginUsed + max_p

df1['qtyAvlbl'] = np.where(df1.marginAvlbl > df1.margin, np.floor(df1.marginAvlbl/df1.margin), 0)

Wall time: 7.79 s


In [28]:
blacklist = list(df1[df1.qtyAvlbl <= 0].index)
remqtydict  = df1[~df1.index.isin(blacklist)].qtyAvlbl.to_dict() # remaining quantity not in blacklist

#**** TO BE DELETED TOMORROW 01-MAR-2019 !!!****
# blacklist = ['JPASSOCIA', 'IDEA']
# remqtydict = pd.DataFrame([])

In [29]:
#...build the high-pop-roc dataframe
fs = listdir('./zdata/')

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle('./zdata/'+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle('./zdata/'+f).reset_index() for f in ohlcs], axis=0, sort=True)
df_und = pd.concat([pd.read_pickle('./zdata/'+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# arrange the columns
cols = ['symbol', 'right', 'expiry', 'dte', 'strike', 'undPrice', 'lo52',  'hi52', 
'stdev', 'volatility', 'margin', 'lot', 'bsmPrice', 'pop', 'rom', 'price', 'option']

df_opt1 = df_opt[cols]

# take only high pops
df_opt2 = df_opt1[df_opt1['pop'] >= min_pop]

df_opt3 = strat_hilo52(df_opt2)

df_opt3 = df_opt3.sort_values('rom', ascending = False)[:150]  # Limit the number of options

# df_opt4 = df_opt3.assign(margin=[short_opt_margin(o, l) for o, l in zip(df_opt3.option, df_opt3.lot)])

df_opt4 = df_opt3

df_opt4 = df_opt4[df_opt4.margin < df_opt4.strike*df_opt4.lot].reset_index(drop=True) # remove 1.7976931348623157e+308 from margins!

# get the better of Price and bsmPrice for the option - with an expected multiple (expmult)
max_price_bsm = pd.concat([df_opt4.price, df_opt4.bsmPrice], axis=1).max(axis=1)
df_opt4['expPrice'] = get_prec(max_price_bsm*expmult, base)

In [35]:
# Make df the dataframe that you want to execute on!
df = df_opt4.copy()   # make this the last dataframe to get the orders placed

df = df.assign(margin_roll=df.margin.expanding(1).sum())
df = df[df.margin_roll < av_funds*2]  # to a max of 2 times available funds

df = df.assign(qty=1) # defaults to a quantity of 1
df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
df = df.assign(rom=df.expPrice/df.margin*365/df.dte*df.lot) # recalculate rom based on expPrice

contracts = [c for c in df.option]

In [36]:
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(contracts), \
      len(df.symbol.unique()), sum(df.margin*df.qty), av_funds*max_nlvp, sum(df.expPrice*df.lot*df.qty)))

96 contracts from 30 scrips, consuming 9,883,037 margin from full available funds of 3,974,509, giving INR 90,348


In [37]:
# ...review calls and puts
desired_rom = 1.8 # desired rom to give the target price.

# add tgtPrice based on desired rom
df['tgtPrice'] = get_prec(pd.concat([df.rom, pd.Series(desired_rom, index=df.index)], axis=1).max(axis=1)*df.expPrice/df.rom, base)

df['remqty'] = round(max_p/df.margin)

cols = ['right', 'symbol', 'strike', 'undPrice', 'dte', 'pop', 'rom', 'price', 'expPrice', 'margin', 'lot', 'qty', 'remqty', 'tgtPrice', 'option']
df = df[cols]

# replace remqty with non-blacklist remaining quantities
df = df.set_index('symbol')
df.remqty = np.where(df.index.isin(remqtydict.keys()), df.index.map(remqtydict), df.remqty)
df = df.reset_index()

# Sort the calls and puts by symbol and strikes - to quickly weed out risky options
gb = df.groupby('right')

if 'C' in [k for k in gb.indices]:
    df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    
    df_callsymbols = df_calls.symbol.unique()
    watchcalls = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df_callsymbols]
    df_wp = util.df(watchcalls)
    df_wp.to_csv('./zdata/callswatch.csv', index=None, header=False)
    
else:
    df_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
    df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    
    # make watchlist
    df_putssymbols = df_puts.symbol.unique()
    watchputs = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df_putssymbols]
    df_wp = util.df(watchputs)
    df_wp.to_csv('./zdata/putswatch.csv', index=None, header=False)
else:
    df_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
df.to_csv('./zdata/check.csv', index=None, header=True)

# .... for puts from path: C:\Users\kashir\Documents\IBKR\nse\zdata\putswatch.csv
# .... or in this path: C:\Users\User\Documents\ibkr\nse\zdata\putswatch.csv (home laptop)


In [38]:
# After going through checked.csv, with puts and calls, eliminate risky options
# Save the file as checked.csv

df_final = pd.read_csv('./zdata/checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=tgtPrice) for lot, tgtPrice in zip(df_final.lot, df_final.tgtPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(cs), \
      len(df_final.symbol.unique()), sum(df_final.margin*df_final.qty), av_funds*max_nlvp, sum(df_final.tgtPrice*df_final.lot*df_final.qty)))

96 contracts from 30 scrips, consuming 33,129,873 margin from full available funds of 3,974,509, giving INR 3,962,232


In [39]:
#----------------------------------------------------
# WARNING: THIS SECTION PLACES OPENING TRADES
#____________________________________________________

openingTrades = [ib.placeOrder(contract, order) for contract, order in zip(cs, orders)]

In [27]:
remqtydict

{'ADANIPORT': 0.0,
 'AMBUJACEM': 1.0,
 'APOLLOTYR': 0.0,
 'ARVIND': 6.0,
 'ASHOKLEY': 3.0,
 'AXISBANK': 1.0,
 'BALKRISIN': 1.0,
 'BANKNIFTY': 4.0,
 'BATAINDIA': 0.0,
 'BEL': 0.0,
 'BHARATFOR': 2.0,
 'BHEL': 1.0,
 'BIOCON': 0.0,
 'BSOFT': 8.0,
 'CANBK': 1.0,
 'CEATLTD': 3.0,
 'CESC': 4.0,
 'COALINDIA': 2.0,
 'CONCOR': 0.0,
 'DHFL': 2.0,
 'DLF': 1.0,
 'ENGINERSI': 3.0,
 'GAIL': 0.0,
 'GLENMARK': 2.0,
 'GSFC': 2.0,
 'HINDALCO': 0.0,
 'HINDZINC': 1.0,
 'IBULHSGFI': 1.0,
 'IDBI': 2.0,
 'IFCI': 1.0,
 'INDIACEM': 3.0,
 'INDIANB': 2.0,
 'JETAIRWAY': 0.0,
 'JISLJALEQ': 0.0,
 'JSWSTEEL': 2.0,
 'KSCL': 0.0,
 'LICHSGFIN': 2.0,
 'LT': 2.0,
 'MCX': 0.0,
 'MFSL': 2.0,
 'MM': 1.0,
 'MMFIN': 2.0,
 'MOTHERSUM': 0.0,
 'MRPL': 2.0,
 'NATIONALU': 1.0,
 'OFSS': 2.0,
 'ORIENTBAN': 2.0,
 'RCOM': 0.0,
 'RELCAPITA': 0.0,
 'RELINFRA': 0.0,
 'REPCOHOME': 3.0,
 'SBIN': 1.0,
 'SIEMENS': 2.0,
 'SUNTV': 1.0,
 'TATAELXSI': 3.0,
 'TATAGLOBA': 0.0,
 'TATASTEEL': 2.0,
 'TITAN': 0.0,
 'TVSMOTOR': 3.0,
 'UNIONBANK': 2.0,
 